# Librerias

In [6]:
#Importar librerias de Scrappy
%pip install scrapy
%pip install spacy
%pip install nltk
#Importar librerias de spacy para español
!python -m spacy download es_core_news_sm
!python -m spacy download es_core_news_md

Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 12.1/12.1 MB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 122.2/122.2 kB ? eta 0:00:00
     ------------------------------------- 395.8/395.8 kB 24.1 MB/s eta 0:00:00
     ---------------------------------------- 181.6/181.6 kB ? eta 0:00:00
     ---------------------------------------- 45.9/45.9 kB ? eta 0:00:00
     ------------------------------------- 481.9/481.9 kB 29.5 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 18.8 MB/s eta 0:00:00
     ---------------------------------------- 49.8/49.8 kB ? eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 43.0 MB/s eta 0:00:00
     ---------------------------------------- 6.6/6.6 MB 35.3 MB/s eta 0:00:00
     ---------------------------------------- 45.0/45.0 kB ? eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extens

     --------------------------------------- 12.9/12.9 MB 13.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
     --------------------------------------- 42.3/42.3 MB 28.5 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [102]:
#Asegurarse que se tienen todas las librerias incluidas
import pandas as pd
import numpy as np
import spacy
import es_core_news_sm
import es_core_news_md
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dgbla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dgbla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Funciones

In [110]:
stop_words = set(stopwords.words('english'))
stop_words.add("\"")
stop_words.add("'")
stop_words.add("’")
stop_words.add("”")
stop_words.add("!")
stop_words.add("“")
stop_words.add("?")

def preProcess(path):
    # LEE 
    with open(path, 'r') as f:
        file = f.readlines()
    
    title = file[0][:-1]
    synopsis = " ".join(file[1::])

    #Minusculas
    lower_text = synopsis.lower()

    #Eliminar signos de puntuación
    nlp = es_core_news_sm.load()
    doc = nlp(lower_text)
    tokens=[]
    # revision de signos de puntuacion y en caso de que no sea se agrega
    for token in doc:
        if token.is_punct==False:
            tokens.append(token.text)

    clean_text = " ".join(tokens)

    #Stopwords
    new_tokens = word_tokenize(clean_text)
    clean_tokens = [w for w in new_tokens if not w in stop_words]
    final_text = " ".join(clean_tokens)
    
    # Guarda
    new_path = path[:53] + '-clean' + path[53:]
    with open(new_path, 'w') as f:
        f.write(f'{title}\n{final_text}')
    
    # returns
    return (int(path[54:-4]), clean_tokens)

In [106]:
df = pd.DataFrame()
def vect(df, id, cleantokens):
    current = list(df.columns)
    c = set(clean_tokens)
    zvect = [0] * df.shape[0]
    for p in c:
        if p not in current:
            df[p] = zvect
            current.append(p)
    d = dict()
    for it in current:
        if it in c:
            sum = clean_tokens.count(it)
            d[it] = sum/len(clean_tokens)
        else:
            d[it] = 0
    new_record = pd.DataFrame([d])
    df = pd.concat([df, new_record], ignore_index=True)
    return df

# Web Scrapping

## Crear el proyecto de Scrappy

In [ ]:
!scrapy startproject GoodReadsScraper
%cd GoodReadsScraper/GoodReadsScraper/spiders/

## Crear un nuevo Spyder

In [5]:
!scrapy genspider GoodReadsSpider https://www.goodreads.com/shelf/show/fiction

C:\Users\Usuario\Documents\University\Recuperacion Web\LibreriaUNAL\Trabajo 2\GoodReadsScraper\GoodReadsScraper\spiders\GoodReadsSpider.py already exists


## Ejecutar el Spyder

In [101]:
!scrapy runspider GoodReadsSpider.py -O ../../ejercicio0.json

2023-10-29 18:14:38 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: GoodReadsScraper)
2023-10-29 18:14:38 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 23.0.0 (OpenSSL 1.1.1t  7 Feb 2023), cryptography 39.0.1, Platform Windows-10-10.0.22621-SP0
2023-10-29 18:14:38 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'GoodReadsScraper',
 'FEED_EXPORT_ENCODING': 'utf-8',
 'NEWSPIDER_MODULE': 'GoodReadsScraper.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_LOADER_WARN_ONLY': True,
 'SPIDER_MODULES': ['GoodReadsScraper.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2023-10-29 18:14:38 [asyncio] DEBUG: Using selector: SelectSelector
2023-10-29 18:14:38 [scrapy.utils.log] DEBUG: Using reactor: twisted.intern

# Pre-procesamiento

## Leer archivo

In [75]:
path = 'C:\\Users\\dgbla\\GitHub\\LibreriaUNAL\\Trabajo 2\\archivos\\4.txt'
with open(path, 'r') as f:
    file = f.readlines()
title = file[0][:-1]
synopsis = " ".join(file[1::])
print(title)
print(synopsis)

To Kill a Mockingbird
The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.


## Poner todo en minuscula

In [76]:
lower_text = synopsis.lower()
print(lower_text)

the unforgettable novel of a childhood in a sleepy southern town and the crisis of conscience that rocked it. "to kill a mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. it went on to win the pulitzer prize in 1961 and was later made into an academy award-winning film, also a classic.


## Eliminar signos de Puntuacion

In [77]:
nlp = es_core_news_sm.load()
doc = nlp(lower_text)
tokens=[]
# revision de signos de puntuacion y en caso de que no sea se agrega
for token in doc:
  if token.is_punct==False:
    tokens.append(token.text)

clean_text = " ".join(tokens)
print(clean_text)

the unforgettable novel of a childhood in a sleepy southern town and the crisis of conscience that rocked it to kill a mockingbird became both an instant bestseller and a critical success when it was first published in 1960 it went on to win the pulitzer prize in 1961 and was later made into an academy award-winning film also a classic


## Eliminar las Stopwords

In [78]:
stop_words = set(stopwords.words('english'))
stop_words.add("\"")
stop_words.add("'")
stop_words.add("’")
stop_words.add("”")
stop_words.add("!")
stop_words.add("“")
stop_words.add("?")

In [79]:
new_tokens = word_tokenize(clean_text)
print(new_tokens)
clean_tokens = [w for w in new_tokens if not w in stop_words]
print(clean_tokens)
final_text = " ".join(clean_tokens)
print(final_text)

['the', 'unforgettable', 'novel', 'of', 'a', 'childhood', 'in', 'a', 'sleepy', 'southern', 'town', 'and', 'the', 'crisis', 'of', 'conscience', 'that', 'rocked', 'it', 'to', 'kill', 'a', 'mockingbird', 'became', 'both', 'an', 'instant', 'bestseller', 'and', 'a', 'critical', 'success', 'when', 'it', 'was', 'first', 'published', 'in', '1960', 'it', 'went', 'on', 'to', 'win', 'the', 'pulitzer', 'prize', 'in', '1961', 'and', 'was', 'later', 'made', 'into', 'an', 'academy', 'award-winning', 'film', 'also', 'a', 'classic']
['unforgettable', 'novel', 'childhood', 'sleepy', 'southern', 'town', 'crisis', 'conscience', 'rocked', 'kill', 'mockingbird', 'became', 'instant', 'bestseller', 'critical', 'success', 'first', 'published', '1960', 'went', 'win', 'pulitzer', 'prize', '1961', 'later', 'made', 'academy', 'award-winning', 'film', 'also', 'classic']
unforgettable novel childhood sleepy southern town crisis conscience rocked kill mockingbird became instant bestseller critical success first publi

## Guardar archivo modificado

In [80]:
path = 'C:\\Users\\dgbla\\GitHub\\LibreriaUNAL\\Trabajo 2\\archivos-clean\\3.txt'
with open(path, 'w') as f:
    f.write(f'{title}\n{final_text}')

# Vectorizacion

## Dataframe of vectors

In [64]:
df = pd.DataFrame()
df

""


In [81]:
current = list(df.columns)

In [82]:
c = set(clean_tokens)

In [83]:
zvect = [0] * df.shape[0]
for p in c:
    if p not in current:
        df[p] = zvect
        current.append(p)

In [87]:
d = dict()
for it in current:
    if it in c:
        sum = clean_tokens.count(it)
        d[it] = sum/len(clean_tokens)
    else:
        d[it] = 0

In [88]:
df

,capitol,ruins,north,panem,annual,keeps,tv,live,one,hunger,...,classic,kill,rocked,childhood,went,mockingbird,success,later,film,bestseller
0,0.054054,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.054054,0.027027,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258


In [89]:
new_record = pd.DataFrame([d])
df = pd.concat([df, new_record], ignore_index=True)
df

,capitol,ruins,north,panem,annual,keeps,tv,live,one,hunger,...,classic,kill,rocked,childhood,went,mockingbird,success,later,film,bestseller
0,0.054054,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.054054,0.027027,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258


# Base de Datos

In [111]:
for p in range(1,11):
    path = 'C:\\Users\\dgbla\\GitHub\\LibreriaUNAL\\Trabajo 2\\archivos\\' + str(p) + '.txt'
    id, clean = preProcess(path)
    df = vect(df, id, clean)
df

,crisis,became,published,pulitzer,award-winning,town,1961,southern,made,instant,...,classic,kill,rocked,childhood,went,mockingbird,success,later,film,bestseller
0,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258
1,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258
2,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258
3,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258
4,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258
5,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258
6,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258
7,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258
8,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258
9,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258


# Consulta